In [5]:
import torch
import numpy as np
import cv2
from time import time
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
drive_path = '/content/drive/MyDrive/DA_CNN기초연습'

In [18]:
class ObjectDetection:

    # YouTube 동영상에 YOLOv5 구현
    def __init__(self, filepath, out_file):
        # 객체 생성 시 호출
        self._filepath = filepath  # url: 예측 대상 YouTube URL
        self.model = self.load_model()
        self.classes = self.model.names
        self.out_file = out_file  # out_file: Detection 출력 파일 이름 *.avi
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'


    def get_video_from_path(self):
        if os.path.isfile(self._filepath):
          cap = cv2.VideoCapture(self._filepath)
        else:
          print('File Not Exist')
        return cap

    def load_model(self):
        # YOLOv5 모델 로드
        model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True) #yolov5 COCO dataset pretrained
        return model

    def score_frame(self, frame):
        # frame: 단일 프레임; numpy/list/tuple 형식
        # return: 프레임에서 모델이 감지한 객체의 레이블과 좌표
        self.model.to(self.device)
        frame = [frame]
        results = self.model(frame)
        labels, cord = results.xyxyn[0][:, -1].cpu().numpy(), results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

    def class_to_label(self, x):
        # x 숫자 레이블 -> 문자열 레이블로 반환
        return self.classes[int(x)]

    def plot_boxes(self, results, frame):
        # 경계상자와 레이블을 프레임에 플로팅
        # results: 프레임에서 모델이 감지한 객체의 레이블과 좌표
        # frame: 점수화된 프레임
        # return: 경계 상자와 레이블이 플로팅된 프레임
        labels, cord = results
        n = len(labels)
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            if row[4] >= 0.2:
                x1, y1, x2, y2 = int(row[0]*x_shape), int(row[1]*y_shape), int(row[2]*x_shape), int(row[3]*y_shape)
                bgr = (0, 255, 0)
                cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
                cv2.putText(frame, self.class_to_label(labels[i])
                            + ': ' + str(x1) + ', ' + str(x2) + ', ' + str(y1) + ', ' + str(y2),
                            (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.9, bgr, 2)
        return frame

    def __call__(self):
        # 인스턴스 생성 시 호출; 프레임 단위로 비디오 로드
        player = self.get_video_from_path()  #cv2.VideoCapture() 객체 생성
        assert player.isOpened() #비디오 열리지 않으면 assert

        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))  #비디오 프레임 가로
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))  #비디오 프레임 세로

        #fourcc(4-문자 코드, four character code) - 동영상 파일의 코덱, 압축 방식, 색상, 픽셀 포맷 등을 정의하는 정수 값
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        fps = 20  #초당 프레임 수

        #cv2.VideoWriter 객체 생성
        out = cv2.VideoWriter(self.out_file, four_cc, fps, (x_shape, y_shape))

        while True:
            start_time = time()
            ret, frame = player.read()

            if not ret:   #비디오가 끝나면 retval = False 출력
              break

            results = self.score_frame(frame)
            frame = self.plot_boxes(results, frame)

            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")

            out.write(frame) #프레임 저장

        cap.release()
        out.release()
        cv2.destroyAllWindows()

In [20]:
a = ObjectDetection(os.path.join(drive_path,'Office-Parkour.mp4'), os.path.join(drive_path,'Office-Parkour_ObjectDetection.avi'))
a()

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-1-2 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Frames Per Second : 31.25
Frames Per Second : 50.0
Frames Per Second : 55.55555555555556
Frames Per Second : 62.5
Frames Per Second : 50.0
Frames Per Second : 47.61904761904761
Frames Per Second : 52.631578947368425
Frames Per Second : 40.0
Frames Per Second : 45.45454545454546
Frames Per Second : 43.47826086956522
Frames Per Second : 55.55555555555556
Frames Per Second : 41.666666666666664
Frames Per Second : 50.0
Frames Per Second : 71.42857142857143
Frames Per Second : 71.42857142857143
Frames Per Second : 71.42857142857143
Frames Per Second : 66.66666666666667
Frames Per Second : 71.42857142857143
Frames Per Second : 71.42857142857143
Frames Per Second : 66.66666666666667
Frames Per Second : 71.42857142857143
Frames Per Second : 58.8235294117647
Frames Per Second : 71.42857142857143
Frames Per Second : 66.66666666666667
Frames Per Second : 71.42857142857143
Frames Per Second : 66.66666666666667
Frames Per Second : 45.45454545454546
Frames Per Second : 66.66666666666667
Frames Per S

In [22]:
cap = cv2.VideoCapture(os.path.join(drive_path,'Office-Parkour.mp4'))
print(f'전체프레임수:{cap.get(cv2.CAP_PROP_FRAME_COUNT)}')

전체프레임수:2293.0


23.976023976023978

In [17]:
while True:
  start_time = time()
  ret, frame = cap.read()
  print(f'현재프레임:{cap.get(cv2.CAP_PROP_POS_FRAMES)}')
  if not ret:
    print("비디오의 끝에 도달하거나 문제 발생")
    break
    # results = self.score_frame(frame)
    # frame = self.plot_boxes(results, frame)
    # end_time = time()
    # fps = 1/np.round(end_time - start_time, 3)
    # print(f"Frames Per Second : {fps}")
    # out.write(frame)

현재프레임:1.0
현재프레임:2.0
현재프레임:3.0
현재프레임:4.0
현재프레임:5.0
현재프레임:6.0
현재프레임:7.0
현재프레임:8.0
현재프레임:9.0
현재프레임:10.0
현재프레임:11.0
현재프레임:12.0
현재프레임:13.0
현재프레임:14.0
현재프레임:15.0
현재프레임:16.0
현재프레임:17.0
현재프레임:18.0
현재프레임:19.0
현재프레임:20.0
현재프레임:21.0
현재프레임:22.0
현재프레임:23.0
현재프레임:24.0
현재프레임:25.0
현재프레임:26.0
현재프레임:27.0
현재프레임:28.0
현재프레임:29.0
현재프레임:30.0
현재프레임:31.0
현재프레임:32.0
현재프레임:33.0
현재프레임:34.0
현재프레임:35.0
현재프레임:36.0
현재프레임:37.0
현재프레임:38.0
현재프레임:39.0
현재프레임:40.0
현재프레임:41.0
현재프레임:42.0
현재프레임:43.0
현재프레임:44.0
현재프레임:45.0
현재프레임:46.0
현재프레임:47.0
현재프레임:48.0
현재프레임:49.0
현재프레임:50.0
현재프레임:51.0
현재프레임:52.0
현재프레임:53.0
현재프레임:54.0
현재프레임:55.0
현재프레임:56.0
현재프레임:57.0
현재프레임:58.0
현재프레임:59.0
현재프레임:60.0
현재프레임:61.0
현재프레임:62.0
현재프레임:63.0
현재프레임:64.0
현재프레임:65.0
현재프레임:66.0
현재프레임:67.0
현재프레임:68.0
현재프레임:69.0
현재프레임:70.0
현재프레임:71.0
현재프레임:72.0
현재프레임:73.0
현재프레임:74.0
현재프레임:75.0
현재프레임:76.0
현재프레임:77.0
현재프레임:78.0
현재프레임:79.0
현재프레임:80.0
현재프레임:81.0
현재프레임:82.0
현재프레임:83.0
현재프레임:84.0
현재프레임:85.0
현재프레임:86.0
현재프레임:87.0
현재프레임:88.0
현재프레임:89.0
현재프레임:90.0
현재프레임:91.0
현재프레임:92